In [ ]:
import nltk
import re
import couchdb
from textblob import TextBlob

In [ ]:
try:
    couch = couchdb.Server('http://115.146.93.140:5984/')
    db = couch['raw_tweets']
except:
    print 'Database Connection Error'

In [ ]:
db['2d368c427b8e1c5c54ca382c290005da'].keys()

In [ ]:
db['2d368c427b8e1c5c54ca382c290005da']['retweeted']

In [ ]:
def processTweet(tweet_text):
    '''Cleans a Tweets by removing unnecessary data'''
    clean_tweet = tweet_text.lower()
    #Convert www.* or https?://* to URL
    clean_tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',clean_tweet)
    #Convert @username to AT_USER
    clean_tweet = re.sub('@[^\s]+','AT_USER',clean_tweet)
    #Remove additional white spaces
    clean_tweet = re.sub('[\s]+', ' ', clean_tweet)
    #Replace #word with word
    clean_tweet = re.sub(r'#([^\s]+)', r'\1', clean_tweet)
    #trim
    clean_tweet = clean_tweet.strip('\'"')
    return clean_tweet

In [ ]:
print tw_text_test
clean_tweet=processTweet(tw_text_test)

In [ ]:
testimonial = TextBlob(clean_tweet)    

In [ ]:
def getSentiment(clean_tweet,neutral_threshold=0):
    '''Return Positive, Neutral or Negative Tweet Rating'''
    blob=TextBlob(clean_tweet).sentiment.polarity
    print blob
    if -neutral_threshold <= blob <= neutral_threshold:
        return "Neutral"
    elif blob>neutral_threshold:
        return "Positive"
    else:
        return "Negative"

In [ ]:
print getSentiment('Trump Sean Spicer et al lie for a reason')

In [ ]:
testimonial = TextBlob('Nashville I ll be heading your way for the very first time for the conference in August Anyone on here live there')  

In [ ]:
testimonial.sentiment

In [ ]:
' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", 'Trump, Sean Spicer, et al. lie for a reason').split())

In [ ]:
' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tw_text_test).split())

In [ ]:
clean_tweet=processTweet(tw_text_test)
print clean_tweet

In [11]:
f1=open('./testfile.log', 'w+')
a=1
print >> f1, "Problem Connecting to API"
#f1.write('This is a test %d' %a)

f1.close()

In [4]:
import sys,os,time
import json,jsonpickle
import tweepy
import couchdb
import TwitterSentiment
#log = open("tnsw.log", "a")
#sys.stdout = log

consumer_key = '6LI8WFTgasVJusVl68WVoJS5X'
consumer_secret = 'GJgYiiOKnNjhyGbsqHFfV4cDOUi7FzQ16DXHc9ca0bBFRi23qv'
access_token = '4506027074-Ics9Dy94Dvs1hxMMJ2OBRJzqGbc9VX1A61OPBFL'
access_secret = 'MWMagt2OsdXBRMVBBtq87OTbbx5NdVlQ55t4zQPWkKrOA'

#Ref: https://www.karambelkar.info/2015/01/how-to-use-twitters-search-rest-api-most-effectively./

#places = api.geo_search(query="AUSTRALIA", granularity="country")

#place_id = places[0].id
#print('AUSTRALIA id is: ',place_id)
#3f14ce28dc7c4566

#Database Setup
try:
    couch = couchdb.Server('http://115.146.93.140:5984/')
    db = couch['raw_tweets']
except:
    print 'Database Connection Error'


auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

#Setting up new api wrapper, using authentication only
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
 
#Error handling
if (not api):
    print ("Problem Connecting to API")

#This is what we are searching for
ailist=['#AI',"artificial intelligence","#deeplearning","datascience",'#robots','robots','automation','#ArtificialIntelligence',
       '#bot','#bots','#ml','#iot','#tech','#analytics']
foodlist=['food','fitness food','#delicious','drink','eat','#coffee','beer','#foodie','#foodvision','#foodtruck','#eatingWell','#eatlocal','#foodparty']
politicslist = ['politics','immigration','migration','visa','#migration','#Australia','permanent migration program',"visa change",'sponsorship australia',
                  'labour party','liberal party','left wing','right wing','liberal party australia',
                  'politics australia','labour party australia','coalition party australia','australian labor party',
                  'australia politics','labour OR liberal','turnbull']

searchList=[ailist,foodlist,politicslist]
#Search Config
maxTweets = 100000000000000
tweetsPerQry = 100
sinceId = None
max_id = -1L

# Search Criteria
lang='en'
geocode='-37.810279,144.962619,5000000mi'


tweetCount = 0


searchListCount=0
totallistcount=0

maxList=len(searchList[searchListCount])
maxlistcount=len(searchList)

print("Downloading max {0} tweets".format(maxTweets))
while tweetCount < maxTweets:
    
    if searchListCount==maxList-1:
        totallistcount+=1
        maxList=len(searchList[searchListCount])

    if totallistcount==maxlistcount-1:
        totallistcount=0
        maxlistcount=len(searchList)

    searchQuery=searchList[totallistcount][searchListCount]
    print "Searching for: ",searchQuery
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang=lang,geocode=geocode)#,)
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang=lang,geocode=geocode,
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang=lang,geocode=geocode,
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,lang=lang,geocode=geocode,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
        if not new_tweets:
            print("No more tweets found, sleeping for 15 minutes")
            time.sleep(15*60)
            searchListCount+=1
            max_id = -1L
            continue
        
        for tweet in new_tweets:
            tweet_ = json.loads(jsonpickle.encode(tweet._json, unpicklable=False))
            clean_tweet_text=TwitterSentiment.processTweet(tweet_['text'])
            sentiment=TwitterSentiment.getSentiment(clean_tweet_text)
            tweet_['sentiment']=sentiment
            tweet_['searchQuery']=searchQuery
            db.save(tweet_)
        
        tweetCount += len(new_tweets)
        print("Downloaded {0} tweets".format(tweetCount))
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("No more tweets found, sleeping for 15 minutes")
        time.sleep(15*60)
        searchListCount+=1
        max_id = -1L
        break

Searching for:  #AI


KeyboardInterrupt: 